# DD142X

### Prerequisites
* Python v3.7
* Tensorflow v1.4.2 
* Installera massa skit med pip för att få det att fungera allteftersom Python klagar.
* Jag ändrade i train_image_classifier.py för att träna på CPU.
* Typ 100 GB hårddiskutrymme.

Det kan funka med andra versioner, men inte Tensorflow 2.

### Ladda in directories
Kör varje gång.

In [1]:
# Set pretrained checkpoint dir.
%env PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained

# Where the training (fine-tuned) checkpoint and logs will be saved to.
%env TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints

# Where the testing results will be saved to.
%env EVAL_DIR=/Users/johan/Dev/kexet/eval

# Where the dataset is saved to.
%env DATASET_DIR=/Users/johan/Dev/kexet/data

env: PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained
env: TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints
env: EVAL_DIR=/Users/johan/Dev/kexet/eval
env: DATASET_DIR=/Users/johan/Dev/kexet/data


### Ladda ner en förtränad inception v3

Kör bara första gången.

In [ ]:
### Download the model if you haven't yet.
!mkdir ${PRETRAINED_CHECKPOINT_DIR}
!wget http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz
!tar -xvf inception_v3_2016_08_28.tar.gz
!mv inception_v3.ckpt ${PRETRAINED_CHECKPOINT_DIR}
!rm inception_v3_2016_08_28.tar.gz

### Organisera ISIC datan

Kör bara första gången.

In [ ]:
import json
import os
import shutil

# Path to the raw ISIC pull.
path_to_data = '/Users/johan/Dev/kexet/rawdata'

# Final data path.
final_path = '/Users/johan/Dev/kexet/data/'


for root, dirs, files in os.walk(path_to_data):
    # Do some basic filtering.
    files = [f for f in files if not f[0] == '.']
    files = [f for f in files if not f[-1] == 'g']
    dirs[:] = [d for d in dirs if not d[0] == '.']
    
    for image_name in files:
        image_path = root + '/' + image_name
        try:
            with open(image_path) as data_file:
                # Read the disease from description file.
                jsonList = json.load(data_file)
                disease = jsonList['meta']['clinical']['diagnosis'].replace(' ', '_')
                data_file.close()

                # Make a directory for storing the data.
                disease_dir = final_path + '/' + disease
                if not os.path.exists(disease_dir):
                    os.makedirs(disease_dir)

                # Copy the image to the new dir.
                image_src_path = path_to_data + '/Images/' + image_name + '.jpeg'
                shutil.copy(image_src_path, disease_dir)

        # Catch exceptions such as wrongly formated images and just skip it.
        except:
            continue
            
print("Finished!")

### Dela in ISIC datan i training/test/validation
Kör bara första gången.

In [ ]:
import os
import shutil
import random

data_dir = "/Users/johan/Dev/kexet/data"

# Over/under sample until we have this many training images of each class.
n_training = 7500

# Target fraction of original images for validation.
frac_validation = 0.1

# Target fraction of original images for testing.
frac_test = 0.1

# List of diseases we are studying with underscore instead of spaces.
diseases = ["melanoma", "nevus", "seborrheic_keratosis"]



# Loop through dirs to remove unwanted classes and create directory structure.
for root, dirs, _ in os.walk(data_dir):
    for dir_name in dirs:
        
        # Remove images of diseases not in disease list.
        if dir_name not in diseases:
            shutil.rmtree(os.path.join(root, dir_name))
            
        # Add test and validation directories.
        else:
            os.mkdir(os.path.join(root, "test_" + dir_name))
            os.mkdir(os.path.join(root, "validation_" + dir_name))
            
        # Prefixa dirs med train_
        os.rename(os.path.join(root, dir_name), os.path.join(root, "train_" + dir_name))         
        
        
# Loop through dirs to move data into test/validation dirs.
for root, dirs, _ in os.walk(data_dir):
    for dir_name in dirs:
        
        # Loop through dirs prefixed train_
        if dir_name.split('_', 1)[0] == "train":
            for inner_root, _, files in os.walk(os.path.join(root, dir_name)):
                
                # Shuffle and then move images to validation/train
                random.shuffle(files)
                
                # Validation/test size
                n_validation = int(round(len(files)*frac_validation))
                n_test = int(round(len(files)*frac_test))
                
                for i in range(n_validation):
                    file = os.path.join(inner_root, files[i])
                    move_to = os.path.join(root, "validation_" + dir_name.split('_', 1)[1])
                    print(file + " -> " + move_to)
                    shutil.move(file, move_to)
                    
                for i in range(n_test):
                    file = os.path.join(inner_root, files[n_validation + i])
                    move_to = os.path.join(root, "test_" + dir_name.split('_', 1)[1])
                    shutil.move(file, move_to)


# Loop through dirs to over/under sample the training data.
for root, dirs, _ in os.walk(data_dir):
    for dir_name in dirs:
        
        # Loop through dirs prefixed train_
        if dir_name.split('_', 1)[0] == "train":
            for inner_root, _, files in os.walk(os.path.join(root, dir_name)):
           
                # Duplicate images in dir if less than n_training.
                n = 0
                while len(files) + n < n_training:
                    for image in files:
                        n = n + 1
                        original = os.path.join(inner_root, image)
                        duplicate = os.path.join(inner_root, "DUPE_" + str(n) + ".jpeg")
                        print(original + " -> " + duplicate)
                        shutil.copy(original, duplicate)
                        if len(files) + n == n_training: break

                print("Added " + str(n) + " to " + dir_name + ". Previously " + str(len(files)) + " pictures.")

            # Remove images in dir if more than n_training.
            for inner_root, _, files in os.walk(os.path.join(root, dir_name)):
                n = 0
                for image in files:
                    if len(files) - n > n_training:
                        n = n + 1
                        print("Removing" + os.path.join(inner_root, image))
                        os.remove(os.path.join(inner_root, image))

                print("Removed " + str(n) + " from " + dir_name + ". Previously " + str(len(files)) + " pictures.")


# Finally loop through to print director summary.
for root, dirs, _ in os.walk(data_dir):
    for dir_name in dirs:
        for inner_root, _, files in os.walk(os.path.join(root, dir_name)):
            print(dir_name + " n=" + str(len(files)))

print("Done!")

### Konvertera datan till Slim format.
Kör bara första gången.

In [2]:
import download_and_convert_cancer as dcc
dcc.run("/Users/johan/Dev/kexet/data")

SyntaxError: invalid syntax (download_and_convert_cancer.py, line 92)

### Transfer learning
* Ladda vikterna från den generella färdigtränade modellen, men exkludera sista lagret med "checkpoint_exclude_scopes". 
* Träna om vikterna i detta lager med "trainable_scopes".

In [ ]:
# Fine-tune only the new layers for 1000 steps.
!python3 train_image_classifier.py \
  --train_dir=${TRAIN_DIR} \
  --dataset_name=cancer \
  --dataset_split_name=train \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3 \
  --checkpoint_path=${PRETRAINED_CHECKPOINT_DIR}/inception_v3.ckpt \
  --checkpoint_exclude_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --trainable_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --max_number_of_steps=1000 \
  --batch_size=100 \
  --learning_rate=0.001 \
  --learning_rate_decay_type=fixed \
  --save_interval_secs=600 \
  --save_summaries_secs=600 \
  --log_every_n_steps=10 \
  --optimizer=rmsprop \
  --weight_decay=0.00004 \
  --opt_epsilon=0.1

In [ ]:


# Run evaluation.
!python3 eval_image_classifier.py \
  --checkpoint_path=/Users/johan/Dev/kexet/checkpoints/ \
  --eval_dir=${EVAL_DIR} \
  --dataset_name=cancer \
  --dataset_split_name=test \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3

In [ ]:
import os
import re
import subprocess

path_to_walk = "/Users/johan/Dev/kexet/checkpoints"

checkpoints = []

for root, dirs, files in os.walk(path_to_walk):
    for file in files:
        valid = re.search('(.*ckpt-\d\d+)\.[^\.]+', file)

        if valid:
            checkpoints.append(path_to_walk + '/' + valid.group(1))

checkpoints = list(dict.fromkeys(checkpoints))
checkpoints.sort()
print(checkpoints)


for checkpoint in checkpoints:   
    checkpoint_string = '--checkpoint_path=' + checkpoint
    print("Evaluating " + checkpoint_string)
    
    process = subprocess.Popen(['python3', 'eval_image_classifier.py',
                                '--dataset_dir=/Users/johan/Dev/kexet/data',
                                '--dataset_split_name=test',
                                '--dataset_name=cancer',
                                '--eval_dir=/Users/johan/Dev/kexet/eval', 
                                '--model_name=inception_v3', 
                                checkpoint_string], 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    stdout, stderr
    print(stdout)
    print(stderr)
    

#                                \
 #     --eval_dir=${EVAL_DIR} \
  #    --dataset_name=cancer \
   #   --dataset_split_name=test \
    #  --dataset_dir=${DATASET_DIR} \
     # --model_name=inception_v3 \
      #--checkpoint_path=' + checkpoint],*/
    





In [ ]:
from preprocessing import inception_preprocessing
import tensorflow as tf

from tensorflow.contrib import slim


def load_batch(dataset, batch_size=32, height=299, width=299, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])
    
    # Preprocess image for usage by Inception.
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels = tf.train.batch(
          [image, image_raw, label],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size)
    
    return images, images_raw, labels

In [ ]:
import numpy as np
import tensorflow as tf
from datasets import cancer
from nets import inception
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.contrib import slim

image_size = inception.inception_v3.default_image_size
batch_size = 4

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.ERROR)
    
    dataset = cancer.get_split('test', '/Users/johan/Dev/kexet/data')
    images, images_raw, labels = load_batch(dataset, height=image_size, width=image_size)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits, _ = inception.inception_v3(images, num_classes=dataset.num_classes, is_training=True)

    probabilities = tf.nn.softmax(logits)
    
    checkpoint_path = tf.train.latest_checkpoint("/Users/johan/Dev/kexet/checkpoints")
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())
    
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)
            np_probabilities, np_images_raw, np_labels = sess.run([probabilities, images_raw, labels])
    
            for i in range(batch_size): 
                image = np_images_raw[i, :, :, :]
                true_label = np_labels[i]
                predicted_label = np.argmax(np_probabilities[i, :])
                predicted_name = dataset.labels_to_names[predicted_label]
                true_name = dataset.labels_to_names[true_label]
                
                plt.figure()
                plt.imshow(image.astype(np.uint8))
                plt.title('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
                plt.axis('off')
                plt.show()

In [ ]:
import os
import shutil
import random

# Target number of training images per category after removing/duping.
n_training = 7500

# Target fraction of original images for validation.
frac_validation = 0.1

# Target fraction of original images for testing.
frac_test = 0.1

# List of diseases.
diseases = ["melanoma", "nevus", "seborrheic keratosis"]




# Loop through dirs.
for root, dirs, _ in os.walk("/Users/johan/Dev/kexet/data"):
    for name in dirs:
        if name not in diseases:
            print(os.path.join(root, name))
            #shutil.rmtree(os.path.join(root, name))
            
        
        """
        # Check for dir prefixed train_
        if name.split('_')[0] == "train":
            for root2, _, files2 in os.walk(os.path.join(root, name)):
                random.shuffle(files2)
                
                for i in range(int(len(files2)*frac_validation)):
                    
                    print (i)

                """
                
            
        
        
        """
        # Duplicate images in dir if less than n_training.
        for root2, _, files2 in os.walk(os.path.join(root, name)):
            n = 0
            while len(files2) + n < n_training:
                for name2 in files2:
                    n = n + 1
                    original = os.path.join(root2, name2)
                    duplicate = os.path.join(root2, "ISIC_" + str(n) + ".jpeg")
                    print("Duplicating " + original + " - " + duplicate)
                    shutil.copy(original, duplicate)
                    
                    if len(files2) + n == n_training:
                        break
            print("Added " + str(n) + " to " + name + ". Previously " + str(len(files2)) + " pictures.")
              
        # Remove images in dir if more than n_training.
        for root2, _, files2 in os.walk(os.path.join(root, name)):
            n = 0
            for name2 in files2:
                if len(files2) - n > n_training:
                    n = n + 1
                    print("Removing " + os.path.join(root2, name2))
                    os.remove(os.path.join(root2, name2))
                    
            print("Removed " + str(n) + " from " + name + ". Previously " + str(len(files2)) + " pictures.")

        
        
        print("Next \n")
        """
        

In [ ]:
"""import os
import shutil

for root, dirs, files in os.walk("/Users/johan/Dev/kexet/data"):
    for name in files:
        print(name + " " + str(len(name)))
        if len(name) != 17:
            original = os.path.join(root, name)
            #new = original + ".jpeg"
            os.remove(original)"""